In [1]:
import pandas as pd
import geonamescache
import unidecode
import re

In [2]:
f = open("data/headlines.txt")
headlines = f.readlines()
f.close()

In [3]:
headlines = [unidecode.unidecode(h.replace("\n","")) for h in headlines]

In [4]:
Headlines = pd.DataFrame(headlines)

In [6]:
Headlines.columns= ["headline"]
Headlines['city'] = ''
Headlines['country'] = ''

In [7]:
gc = geonamescache.GeonamesCache()

In [8]:
countries = gc.get_countries()

In [9]:
cities = gc.get_cities()

In [10]:
CC = pd.DataFrame([(cc,unidecode.unidecode(countries[cc]["name"])) for cc in countries])

In [11]:
CC.columns=("countrycode","country")

In [15]:
CC.head()

,countrycode,country
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Antigua and Barbuda
4,AI,Anguilla


In [12]:
C = pd.DataFrame([(c,unidecode.unidecode(cities[c]["name"]), cities[c]["countrycode"]) for c in cities])

In [13]:
C.columns = ("id", "city", "countrycode")

In [14]:
C.head()

,id,city,countrycode
0,3041563,Andorra la Vella,AD
1,290594,Umm Al Quwain City,AE
2,291074,Ras Al Khaimah City,AE
3,291580,Zayed City,AE
4,291696,Khawr Fakkan,AE


In [16]:
for cc in CC.country:
    for i,h in Headlines[Headlines.headline.str.contains(r'\b'+cc+'\\b')].iterrows():
        Headlines.iloc[i]['country'] = cc

In [18]:
for c in C.city:
    for i,h in Headlines[Headlines.headline.str.contains(r'\b'+c+'\\b')].iterrows():
        Headlines.iloc[i]['city'] = c

C:\Users\pborg\AppData\Local\Continuum\anaconda3\envs\discovering-disease-outbreaks\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [19]:
Headlines.head()

,headline,city,country
0,Zika Outbreak Hits Miami,Miami,
1,Could Zika Reach New York City?,New York City,
2,First Case of Zika in Miami Beach,Miami Beach,
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
4,Dallas man comes down with case of Zika,Dallas,


In [20]:
H = pd.merge(Headlines,C,on='city',how='left')

In [22]:
H.drop(['id'], axis=1,inplace=True)

In [25]:
Headlines.shape

(650, 3)

In [24]:
H.shape

(1016, 4)

    Number of lines in H > number of headlines since a number of towns have the same name across countries (.eg. Richmond, Monroe...)

In [27]:
C[ (C.city=='Monroe') | (C.city=='Richmond') ].groupby(['city', 'countrycode']).count()

id
city     countrycode    
Monroe   US            4
Richmond AU            1
         CA            1
         US            4
         ZA            1

In [28]:
H.groupby(['city']).count().reset_index().sort_values('headline', ascending=False).head()

,city,headline,country,countrycode
0,,42,42,0
331,Monroe,16,16,16
414,Richmond,14,14,14
368,Orange,10,10,10
6,Albany,10,10,10
